In [1]:
import numpy as np
import pandas as pd
import time
import itertools
import glob
import json
from scipy.spatial.distance import squareform, pdist
from math import radians, cos, sin, asin, sqrt
from keplergl import KeplerGl
import keplergl

#%matplotlib notebook
# Install keplergl : use "pip install keplergl"
# Also, make sure you have the 'map.json' file for the configeration of the map

# GPS Loon Data

In [2]:
#get data
df = pd.read_csv('loon-flights-2021Q1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6443882 entries, 0 to 6443881
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   flight_id               object 
 1   time                    object 
 2   latitude                float64
 3   longitude               float64
 4   altitude                float64
 5   temperature             float64
 6   pressure                float64
 7   earth_ir                float64
 8   earth_ir_sensor_config  int64  
 9   acs                     int64  
 10  propeller_on            int64  
 11  velocity_u              float64
 12  velocity_v              float64
 13  omega                   float64
 14  acceleration            float64
 15  solar_elevation         float64
 16  solar_azimuth           float64
 17  is_daytime              int64  
dtypes: float64(12), int64(4), object(2)
memory usage: 884.9+ MB


### Clean Data

In [3]:
df= df[df['acs'] == 0] #only keep entries with acs = 0 
df= df[df['altitude'].notnull()] #want to keep entries of balloons in the air

In [4]:
#now only keep temperature, altitude, longitude, latitude, time, and flight id
clmns = ['temperature', 'altitude', 'longitude', 'latitude', 'time', 'flight_id']
df = df[df.columns[df.columns.isin(clmns)]]
df.head()

,flight_id,time,latitude,longitude,altitude,temperature
57,LN-127,2021-01-01T00:46:52.000Z,3.762977,-76.406335,18300.0,199.71
58,LN-127,2021-01-01T00:47:31.000Z,3.759985,-76.403821,18324.5,NaN
59,LN-127,2021-01-01T00:47:52.000Z,3.758394,-76.402441,18315.0,200.48
60,LN-127,2021-01-01T00:48:53.000Z,3.753812,-76.398384,18321.0,200.07
61,LN-127,2021-01-01T00:49:53.000Z,3.749426,-76.394341,18323.2,199.74


In [5]:
#before dropping NaN values convert the time column to datetime object
df['time_dt'] = pd.to_datetime(df['time'])
df.head()

,flight_id,time,latitude,longitude,altitude,temperature,time_dt
57,LN-127,2021-01-01T00:46:52.000Z,3.762977,-76.406335,18300.0,199.71,2021-01-01 00:46:52+00:00
58,LN-127,2021-01-01T00:47:31.000Z,3.759985,-76.403821,18324.5,NaN,2021-01-01 00:47:31+00:00
59,LN-127,2021-01-01T00:47:52.000Z,3.758394,-76.402441,18315.0,200.48,2021-01-01 00:47:52+00:00
60,LN-127,2021-01-01T00:48:53.000Z,3.753812,-76.398384,18321.0,200.07,2021-01-01 00:48:53+00:00
61,LN-127,2021-01-01T00:49:53.000Z,3.749426,-76.394341,18323.2,199.74,2021-01-01 00:49:53+00:00


In [6]:
#work with a smaller date range
start_date = '2021-01-01'
end_date = '2021-01-18'
df = df[(df['time_dt'] >= pd.to_datetime(start_date, utc=True)) & (df['time_dt'] <= pd.to_datetime(end_date, utc=True))]

In [7]:
#remove NaN temperature values 
df= df[df['temperature'].notnull()] #want to keep entries of balloons in the air
print(df['temperature'].isnull().sum())

0


In [8]:
#check how many flights are in date range 
allfltids = df['flight_id'].unique().tolist() #legend 
print(len(allfltids))

69


In [9]:
#check number of flight entries 
flt_filt = df.groupby(['flight_id']).agg({'time_dt':[np.min,np.max,'nunique']}).reset_index().rename(columns = {'amin':'start_time','amax':'end_time','nunique':'num_entries'})

#remove flights with low number of entries -- keep flights with over 20k entries
flt_filt = flt_filt[flt_filt["time_dt"]["num_entries"]>20000]

#check number of remaining flights to work with
allfltids = flt_filt['flight_id'].unique().tolist()
print(len(allfltids))

53


In [10]:
#keep only flights in allfltids
df = df[df['flight_id'].isin(allfltids)]

In [11]:
#check above worked -- don't need to run this cell 
check_fltids = df['flight_id'].unique().tolist()
print(len(check_fltids))

53


### Resample dataset so time indices match

In [12]:
#resample all of the flights -- we need to go flight by flight so we don't account for the jump between two different flights

#empty dataset
clmns = list(df.columns.values)
df_resamp = pd.DataFrame(columns=clmns)

for flt_id in allfltids: 
    df_temp = df[df['flight_id'] == flt_id]
    df_temp = df_temp.set_index('time_dt').resample('1T').ffill() #resample every 1 min -- set index removes time_dt as a column
    df_temp= df_temp[df_temp['flight_id'].notnull()] #get rid of 1st NaN value from resample
   
    #add new time idx back for later use
    time_idx = df_temp.index
    df_temp['time_dt'] = time_idx
    
    #concat to new dataframe
    df_resamp = pd.concat([df_resamp, df_temp], sort = False)

In [13]:
#reaggregate the resampled dataset 
flt_filt = df_resamp.groupby(['flight_id']).agg({'time_dt':[np.min,np.max,'nunique']}).reset_index().rename(columns = {'amin':'start_time','amax':'end_time','nunique':'num_entries'})

len(flt_filt['flight_id'].unique().tolist())

53

In [14]:
#find  min number of entries 
min_entries = flt_filt['time_dt']['num_entries'].min()
print(min_entries)

23384


In [15]:
#get flight associated with min entries -- use the start and end time to further filter the flights 
idx = flt_filt.index[flt_filt['time_dt']['num_entries'] == min_entries].tolist()
idx = idx[0]

In [16]:
#get start and end time 
flight = flt_filt.iloc[idx]
start_time = flight['time_dt']['start_time']
print(start_time)
end_time = flight['time_dt']['end_time']
print(end_time)

2021-01-01 00:01:00+00:00
2021-01-17 05:44:00+00:00


In [17]:
#need to truncate the df_resamp flight to stop at end_time 
df_resamp = df_resamp[(df_resamp['time_dt'] >= pd.to_datetime(start_time)) & (df_resamp['time_dt'] <= pd.to_datetime(end_time))]
df_resamp.head()

,flight_id,time,latitude,longitude,altitude,temperature,time_dt
2021-01-01 00:01:00+00:00,LN-128,2021-01-01T00:00:49.000Z,-4.019081,-78.979806,18776.8,204.03,2021-01-01 00:01:00+00:00
2021-01-01 00:02:00+00:00,LN-128,2021-01-01T00:01:50.000Z,-4.017329,-78.983058,18779.1,204.52,2021-01-01 00:02:00+00:00
2021-01-01 00:03:00+00:00,LN-128,2021-01-01T00:02:50.000Z,-4.01566,-78.986056,18780.3,205.2,2021-01-01 00:03:00+00:00
2021-01-01 00:04:00+00:00,LN-128,2021-01-01T00:03:50.000Z,-4.013952,-78.989027,18775.4,205.28,2021-01-01 00:04:00+00:00
2021-01-01 00:05:00+00:00,LN-128,2021-01-01T00:04:51.000Z,-4.012111,-78.992135,18778.4,204.15,2021-01-01 00:05:00+00:00


In [18]:
#check start and end time filtering worked 
flt_filt = df_resamp.groupby(['flight_id']).agg({'time_dt':[np.min,np.max,'nunique']}).reset_index().rename(columns = {'amin':'start_time','amax':'end_time','nunique':'num_entries'})

print(flt_filt['time_dt']['num_entries'])

0     23384
1     23384
2     23384
3     23384
4     23384
5     23379
6     23384
7     23384
8     23384
9     23384
10    23384
11    23348
12    23384
13    23306
14    23353
15    23384
16    23384
17    23384
18    23384
19    23381
20    23384
21    23384
22    23384
23    23384
24    23384
25    23384
26    23360
27    23384
28    23384
29    23362
30    23384
31    23384
32    23364
33    23359
34    23384
35    23349
36    23384
37    23384
38    23350
39    23384
40    23384
41    23384
42    23382
43    23384
44    23384
45    23357
46    23328
47    23384
48    23384
49    23383
50    23384
51    23384
52    23384
Name: num_entries, dtype: int64


In [19]:
#filter any flights that don't have the same number of entries 
flt_filt = flt_filt[flt_filt["time_dt"]["num_entries"] == min_entries]

print(flt_filt) 
#everything has the same start and end time so the next cell should run fine

   flight_id                    time_dt                                       
                             start_time                   end_time num_entries
0     LN-128  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
1     LN-172  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
2     LN-177  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
3     LN-179  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
4     LN-180  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
6     LN-189  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
7     LN-190  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
8     LN-191  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
9     LN-192  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
10    LN-193  2021-01-01 00:01:00+00:00  2021-01-17 05:44:00+00:00       23384
12    LN-197  2021-01-01 00:01:00+00:00  2021-01-17 

In [20]:
#filter df_resamp 
flt_ids = flt_filt['flight_id'].unique().tolist()
df_resamp = df_resamp[df_resamp['flight_id'].isin(flt_ids)]
df = df_resamp.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 888592 entries, 2021-01-01 00:01:00+00:00 to 2021-01-17 05:44:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   flight_id    888592 non-null  object
 1   time         888592 non-null  object
 2   latitude     888592 non-null  object
 3   longitude    888592 non-null  object
 4   altitude     888592 non-null  object
 5   temperature  888592 non-null  object
 6   time_dt      888592 non-null  object
dtypes: object(7)
memory usage: 54.2+ MB


In [21]:
# Filter data to only incude GPS data. 
df_loon = df[['time_dt','flight_id','latitude', 'longitude', 'altitude']]
df_loon.head()

,time_dt,flight_id,latitude,longitude,altitude
2021-01-01 00:01:00+00:00,2021-01-01 00:01:00+00:00,LN-128,-4.019081,-78.979806,18776.8
2021-01-01 00:02:00+00:00,2021-01-01 00:02:00+00:00,LN-128,-4.017329,-78.983058,18779.1
2021-01-01 00:03:00+00:00,2021-01-01 00:03:00+00:00,LN-128,-4.01566,-78.986056,18780.3
2021-01-01 00:04:00+00:00,2021-01-01 00:04:00+00:00,LN-128,-4.013952,-78.989027,18775.4
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,LN-128,-4.012111,-78.992135,18778.4


# Link Data

In [22]:
df = pd.read_csv('link_intents.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44942562 entries, 0 to 44942561
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   timestamp           object
 1    identifier         object
 2    node_a_identifier  object
 3    node_a_interface   object
 4    node_b_identifier  object
 5    node_b_interface   object
 6    state              object
dtypes: object(7)
memory usage: 2.3+ GB


## Check Time Stamps
The link intents file is quite large. Excel will not load the dataset entirely. At first glance, it appears that there are no dates associated with the timestamps. We want to check if this is true. First, convert the timestamp column to a date time object. Then, check if any of the entries end in a year. If so, we can then filter to the same date range as the gps data and see what links the TS-SDN selected. 

In [23]:
df['time_dt_intents'] = pd.to_datetime(df['timestamp']) #convert to date time
df['time_dt_intents']

0          2018-11-26 09:56:07.789025
1          2018-11-26 09:56:07.928088
2          2018-11-26 10:01:30.578018
3          2018-11-26 10:01:30.834429
4          2018-11-26 10:01:30.954537
                      ...            
44942557   2021-03-01 17:42:04.206962
44942558   2021-03-01 17:42:04.363790
44942559   2021-03-01 17:42:04.382646
44942560   2021-03-01 17:42:05.161701
44942561   2021-03-01 17:42:05.189539
Name: time_dt_intents, Length: 44942562, dtype: datetime64[ns]

In [24]:
#the timestamp is on the order of a nano second
#for filtering we care only about the date 

df['time_dt_intents_date'] = pd.to_datetime(df['timestamp']).dt.date
df['time_dt_intents_date']

0           2018-11-26
1           2018-11-26
2           2018-11-26
3           2018-11-26
4           2018-11-26
               ...    
44942557    2021-03-01
44942558    2021-03-01
44942559    2021-03-01
44942560    2021-03-01
44942561    2021-03-01
Name: time_dt_intents_date, Length: 44942562, dtype: object

In [25]:
df = df[(df['time_dt_intents_date'] >= pd.to_datetime(start_date, utc=True)) & (df['time_dt_intents_date'] <= pd.to_datetime(end_date, utc=True))]
#filter furter to start and stop time 

start_time = '2021-01-01 00:01:00.000000'
stop_time = '2021-01-17 05:44:00.000000'

df = df[(df['time_dt_intents'] >= pd.to_datetime(start_time, utc=False)) & (df['time_dt_intents'] <= pd.to_datetime(stop_time, utc=False))]

c:\programdata\miniconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


## Filter Link Selections 
We want to get the balloon to balloon pairings that the TS-SDN selected. Then we can use these link selections as a baseline for comparing our algorithm to. 

In [26]:
df.head(10)

,timestamp,identifier,node_a_identifier,node_a_interface,node_b_identifier,node_b_interface,state,time_dt_intents,time_dt_intents_date
42764422,2021-01-01 00:01:05.102763,Temporospatial Topology & Routing:Intent:-922...,lwg01.nrr01,wifi0,LN-316,wifi0,INSTALLING,2021-01-01 00:01:05.102763,2021-01-01
42764423,2021-01-01 00:01:05.272345,Temporospatial Topology & Routing:Intent:-922...,lwg01.nrr01,wifi0,LN-316,wifi0,FAILED,2021-01-01 00:01:05.272345,2021-01-01
42764424,2021-01-01 00:01:05.401223,Temporospatial Topology & Routing:Intent:-922...,lwg01.nrr01,wifi0,LN-316,wifi0,INSTALL_REQ,2021-01-01 00:01:05.401223,2021-01-01
42764425,2021-01-01 00:01:07.760114,Temporospatial Topology & Routing:Intent:-922...,LN-065,wifi0,lwg01.wmc03,wifi0,INSTALLING,2021-01-01 00:01:07.760114,2021-01-01
42764426,2021-01-01 00:01:07.909056,Temporospatial Topology & Routing:Intent:-922...,LN-065,wifi0,lwg01.wmc03,wifi0,FAILED,2021-01-01 00:01:07.909056,2021-01-01
42764427,2021-01-01 00:01:09.364512,Temporospatial Topology & Routing:Intent:-922...,LN-065,wifi0,lwg01.wmc03,wifi0,INSTALL_REQ,2021-01-01 00:01:09.364512,2021-01-01
42764428,2021-01-01 00:01:12.044226,Temporospatial Topology & Routing:Intent:-922...,lwg03.wmc03,wifi0,LN-183,wifi0,INSTALLING,2021-01-01 00:01:12.044226,2021-01-01
42764429,2021-01-01 00:01:12.211053,Temporospatial Topology & Routing:Intent:-922...,lwg03.wmc03,wifi0,LN-183,wifi0,FAILED,2021-01-01 00:01:12.211053,2021-01-01
42764430,2021-01-01 00:01:13.458894,Temporospatial Topology & Routing:Intent:-922...,lwg03.wmc03,wifi0,LN-183,wifi0,INSTALL_REQ,2021-01-01 00:01:13.458894,2021-01-01
42764431,2021-01-01 00:01:13.567116,Temporospatial Topology & Routing:Intent:-922...,LN-274,wifi0,lwg02.wmc03,wifi0,INSTALLING,2021-01-01 00:01:13.567116,2021-01-01


Now we want to filter out the node identifiers for the ground stations (ie: lwg03.wmc03)

In [27]:
vals = np.logical_not(df[' node_a_identifier'].str.contains('lwg'))
df = df[vals]
vals = np.logical_not(df[' node_b_identifier'].str.contains('lwg'))
df = df[vals]
df.head()

,timestamp,identifier,node_a_identifier,node_a_interface,node_b_identifier,node_b_interface,state,time_dt_intents,time_dt_intents_date
42764434,2021-01-01 00:01:18.176876,mmwprober: LN-318 LN-318/mmwave1 1609458805: ...,LN-318,mmwave1,LN-320,mmwave2,FAILED,2021-01-01 00:01:18.176876,2021-01-01
42764435,2021-01-01 00:01:18.367363,mmwprober: LN-318 LN-318/mmwave1 1609458805: ...,LN-318,mmwave1,LN-320,mmwave2,INSTALL_REQ,2021-01-01 00:01:18.367363,2021-01-01
42764436,2021-01-01 00:01:18.549323,mmwprober: LN-318 LN-318/mmwave1 1609458805: ...,LN-318,mmwave1,LN-320,mmwave2,INSTALLING,2021-01-01 00:01:18.549323,2021-01-01
42764473,2021-01-01 00:02:19.928041,Temporospatial Topology & Routing:Intent:-922...,,mmwave0,LN-232,mmwave2,FAILED,2021-01-01 00:02:19.928041,2021-01-01
42764474,2021-01-01 00:02:20.983943,Temporospatial Topology & Routing:Intent:-922...,,mmwave0,LN-232,mmwave2,INSTALL_REQ,2021-01-01 00:02:20.983943,2021-01-01


In [28]:
df = df[df[' node_a_identifier'].str.strip().astype(bool)]
df = df[df[' state'] == ' INSTALL_REQ']
df = df[['time_dt_intents', ' node_a_identifier', ' node_b_identifier' ]]
df_link = df
df_link.head()

,time_dt_intents,node_a_identifier,node_b_identifier
42764435,2021-01-01 00:01:18.367363,LN-318,LN-320
42764513,2021-01-01 00:03:17.424535,LN-263,LN-223
42764567,2021-01-01 00:04:40.518581,LN-263,LN-223
42764636,2021-01-01 00:06:32.972957,LN-263,LN-223
42764657,2021-01-01 00:07:01.046237,LN-318,LN-320


# Loon and Link Data Merge 

In [29]:
# Convert the time columns to datetime
df_link['time_dt_intents'] = pd.to_datetime(df_link['time_dt_intents']).dt.tz_localize('UTC')
df_loon['time_dt'] = pd.to_datetime(df_loon['time_dt'])

# Find the min time
min_time = min(df_link['time_dt_intents'].min(), df_loon['time_dt'].min())

# Convert time data to minutes integer value
df_link['time_minutes'] = df_link['time_dt_intents'].apply(lambda x: round((x - min_time).total_seconds() / 60))
df_loon['time_minutes'] = df_loon['time_dt'].apply(lambda x: round((x - min_time).total_seconds() / 60))

# Check that all 'time_minutes' values in link_intents_df are present in loon_data_df
check_time_values = all(time_value in df_loon['time_minutes'].values for time_value in df_link['time_minutes'].values)

print(f"All time values in 'df_link' are present in 'loon_data_df': {check_time_values}")

All time values in 'df_link' are present in 'loon_data_df': True


In [30]:

# Delete spaces
df_link['node_a_identifier'] = df_link[' node_a_identifier'].str.strip()
df_link['node_b_identifier'] = df_link[' node_b_identifier'].str.strip()

# Merge df_links with df_locations on 'node_a_identifier' and 'time_minutes'
df_link = pd.merge(df_link, df_loon[['time_minutes', 'flight_id', 'latitude', 'longitude']], 
                    left_on=['time_minutes', 'node_a_identifier'], 
                    right_on=['time_minutes', 'flight_id'], 
                    how='left')

# Rename the new 'latitude' and 'longitude' columns
df_link.rename(columns={'latitude': 'node_a_latitude', 'longitude': 'node_a_longitude'}, inplace=True)
df_link.drop(columns='flight_id', inplace=True)

# Merge df_links with df_locations on 'node_b_identifier' and 'time_minutes'
df_link = pd.merge(df_link, df_loon[['time_minutes', 'flight_id', 'latitude', 'longitude']], 
                    left_on=['time_minutes', 'node_b_identifier'], 
                    right_on=['time_minutes', 'flight_id'], 
                    how='left')

# Rename the new 'latitude' and 'longitude' columns
df_link.rename(columns={'latitude': 'node_b_latitude', 'longitude': 'node_b_longitude'}, inplace=True)
df_link.drop(columns=['flight_id', ' node_a_identifier',' node_b_identifier'], inplace=True)

# Display the first few rows of the updated DataFrame
df_link.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45056 entries, 0 to 45055
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   time_dt_intents    45056 non-null  datetime64[ns, UTC]
 1   time_minutes       45056 non-null  int64              
 2   node_a_identifier  45056 non-null  object             
 3   node_b_identifier  45056 non-null  object             
 4   node_a_latitude    27312 non-null  object             
 5   node_a_longitude   27312 non-null  object             
 6   node_b_latitude    25720 non-null  object             
 7   node_b_longitude   25720 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 3.1+ MB


In [31]:
df_link.head(1000)

,time_dt_intents,time_minutes,node_a_identifier,node_b_identifier,node_a_latitude,node_a_longitude,node_b_latitude,node_b_longitude
0,2021-01-01 00:01:18.367363+00:00,0,LN-318,LN-320,11.630319,11.469931,12.234865,11.301476
1,2021-01-01 00:03:17.424535+00:00,2,LN-263,LN-223,-26.247413,7.565259,-27.374754,11.161327
2,2021-01-01 00:04:40.518581+00:00,4,LN-263,LN-223,-26.251917,7.572338,-27.380905,11.179544
3,2021-01-01 00:06:32.972957+00:00,6,LN-263,LN-223,-26.260918,7.586412,-27.386777,11.198157
4,2021-01-01 00:07:01.046237+00:00,6,LN-318,LN-320,11.604284,11.504774,12.226036,11.327122
...,...,...,...,...,...,...,...,...
995,2021-01-01 05:27:07.578081+00:00,326,LN-318,LN-312,11.007295,12.58734,12.820759,11.450489
996,2021-01-01 05:28:20.270092+00:00,327,LN-297,LN-293,2.979367,31.553165,NaN,NaN
997,2021-01-01 05:28:27.969271+00:00,327,LN-296,LN-297,NaN,NaN,2.979367,31.553165
998,2021-01-01 05:28:37.030741+00:00,328,LN-197,LN-309,3.54801,37.923389,1.154497,34.812914


In [32]:
df_loon.head()

,time_dt,flight_id,latitude,longitude,altitude,time_minutes
2021-01-01 00:01:00+00:00,2021-01-01 00:01:00+00:00,LN-128,-4.019081,-78.979806,18776.8,0
2021-01-01 00:02:00+00:00,2021-01-01 00:02:00+00:00,LN-128,-4.017329,-78.983058,18779.1,1
2021-01-01 00:03:00+00:00,2021-01-01 00:03:00+00:00,LN-128,-4.01566,-78.986056,18780.3,2
2021-01-01 00:04:00+00:00,2021-01-01 00:04:00+00:00,LN-128,-4.013952,-78.989027,18775.4,3
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,LN-128,-4.012111,-78.992135,18778.4,4


In [33]:
#Drop time columns 
df_loon = df_loon.drop(columns=['time_dt'])
df_link = df_link.drop(columns=['time_dt_intents'])

In [34]:
#Merge both data frames into one df
df = pd.concat([df_loon, df_link], ignore_index=True)

In [35]:
df.head(900001)

,flight_id,latitude,longitude,altitude,time_minutes,node_a_identifier,node_b_identifier,node_a_latitude,node_a_longitude,node_b_latitude,node_b_longitude
0,LN-128,-4.019081,-78.979806,18776.8,0,NaN,NaN,NaN,NaN,NaN,NaN
1,LN-128,-4.017329,-78.983058,18779.1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,LN-128,-4.01566,-78.986056,18780.3,2,NaN,NaN,NaN,NaN,NaN,NaN
3,LN-128,-4.013952,-78.989027,18775.4,3,NaN,NaN,NaN,NaN,NaN,NaN
4,LN-128,-4.012111,-78.992135,18778.4,4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
899996,NaN,NaN,NaN,NaN,4445,LN-287,LN-236,-20.127118,67.575774,NaN,NaN
899997,NaN,NaN,NaN,NaN,4447,LN-263,LN-192,-24.334484,48.061995,-27.321098,50.081679
899998,NaN,NaN,NaN,NaN,4448,LN-287,LN-236,-20.121719,67.573097,NaN,NaN
899999,NaN,NaN,NaN,NaN,4449,LN-297,LN-289,1.488411,54.082315,1.278736,54.246377


In [36]:
# Convert to time
df['time_minutes'] = pd.to_timedelta(df['time_minutes'], unit='m')

# Add 'time_minutes' to 'min_time' to get the actual time
df['time'] = min_time + df['time_minutes']
df['time'] = df['time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

# Drop time minuet column 
df = df.drop(columns=['time_minutes'])

In [37]:
df.head(90000)

,flight_id,latitude,longitude,altitude,node_a_identifier,node_b_identifier,node_a_latitude,node_a_longitude,node_b_latitude,node_b_longitude,time
0,LN-128,-4.019081,-78.979806,18776.8,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:01:00
1,LN-128,-4.017329,-78.983058,18779.1,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:02:00
2,LN-128,-4.01566,-78.986056,18780.3,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:00
3,LN-128,-4.013952,-78.989027,18775.4,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:04:00
4,LN-128,-4.012111,-78.992135,18778.4,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...
89995,LN-179,-9.729919,73.986815,17065.9,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-14 18:44:00
89996,LN-179,-9.732876,73.984582,17077.8,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-14 18:45:00
89997,LN-179,-9.732876,73.984582,17077.8,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-14 18:46:00
89998,LN-179,-9.735904,73.982287,17080.9,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-14 18:47:00


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933648 entries, 0 to 933647
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   flight_id          888592 non-null  object
 1   latitude           888592 non-null  object
 2   longitude          888592 non-null  object
 3   altitude           888592 non-null  object
 4   node_a_identifier  45056 non-null   object
 5   node_b_identifier  45056 non-null   object
 6   node_a_latitude    27312 non-null   object
 7   node_a_longitude   27312 non-null   object
 8   node_b_latitude    25720 non-null   object
 9   node_b_longitude   25720 non-null   object
 10  time               933648 non-null  object
dtypes: object(11)
memory usage: 78.4+ MB


# Map with links

In [51]:
map_1 = keplergl.KeplerGl()
map_1.add_data(data=df, name='Loon Data')

with open('map.json', 'r') as f:
    config = json.load(f)
map_1.config = config
map_1

# On map, click button on top-left to get a sidebar in the left 
# Then click the funnel (filters) and click the clock (Time Playback) on the time_dt section 
# Close the left side bar and a slider should pop up below 
# Click on the play button in the bottom left for the animation 
# Load the map.json config file
# Note NaN values are not displayed

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['Loon Data'], 'id': 'if0474k…

In [52]:
# Save the HTML file
map_1.save_to_html(file_name="map.html", config=config)

Map saved to map.html!


In [50]:
# Only run this code if you want to make changes to the map and save it
# Make changes to the map above and run this code to save config
with open('map.json', 'w') as f:
    json.dump(map_1.config, f)
